In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pandas import DataFrame, ExcelFile, read_csv, read_excel

C:\D\Installation\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df=pd.read_csv('../Jupyter/others/profiles1.csv')

In [3]:
df.head()

,Position Title:,Department:,responsabilities,POSITION SUMMARY:,responsibilities,Date Revised: (Select today’s date),CA,CP,EI,ER,TS
0,Environmental Engineer/Specialist,Technologists (US),"Manage the permitting actions for the RCRA Hazardous Waste Installation and Operation Permit. . Manage the Corrective Actions Program/Site Remediation (including the Soil Management Program. . Supervise the waste handling contractors (including the internal movement of waste in containers/tanks), and possibly the Leak Detection and Repair (LDAR) contractors. . Function as the Oil Spill Director for the site, with responsibility for keeping the Integrated Contingency Plan current and conducting all identified drills. . Oversee the Stormwater Management Plan and the Biosolids Management Plan. . Manage the permitting process for the Deepwell Injection Project. . Support refinery compliance with applicable environmental regulation and permit requirements for the waste management program for the refinery. . Perform emissions and discharge calculations, completing regulatory compliance reports and coordinating environmental sampling and testing activities. . Assist in adapting p...","The Lima Refining Company (LRC) owns and operates a world scale oil refinery located in Lima, Ohio that produces a wide range of transportation fuels and petrochemical feedstocks. LRC is wholly owned subsidiary of Husky Oil Operations Limited, which is one of Canada's largest integrated energy companies. Husky is headquartered in Calgary, Alberta, is publicly traded on the Toronto Stock Exchange, and operates worldwide with Upstream, Midstream, and Downstream business segments. Our Lima, Ohio Refinery has a challenging and rewarding opportunity available for an Environmental Engineer/Specialist. The Environmental Engineer/Specialist will be responsible for the oversight, implementation and compliance with all permit terms and conditions of the waste management programs. This includes the monitoring, record-keeping, inspection, testing, and reporting associated with each of the programs listed below. SPECIFIC RESPONSIBILITIES INCLUDE: - Manage the permitting actions for the R...",NaN,2016-01-04 08:22:52,"BS in Chemical Engineering, Environmental Engineering or other technical degree$$$%%%&&&Master's degree in Engineering or MBA",Professional Engineers License,Minimum of 2 years environmental experience in the petrochemical industry,"Minimum of 2 years of experience as an Environmental Engineer/Specialist$$$%%%&&&or other relevant experience The Lima Refining Company is an Equal Opportunity Employer. All qualified applicants will receive consideration for employment without regard to race, color, religion, sex national origin, veteran status, disability, sexual orientation or gender identity.","Proficient in use of Excel, Word and Air Services$$$%%%&&&Ability to negotiate with regulatory agencies Assets:"
1,"Sr. Staff Engineer, Capital Efficiency",Engineers,"Portfolio management . coordinate and support BU capital planning efforts – enforce standards, check quality, analyze individual projects. Ensure project preparedness of AFE requests and package AFE requests for presentation to the SVP, Downstream for approval. . Act as gatekeeper for turnaround preparedness prior to full funding; optimize uptime by coordinating between Downstream assets . Establish the processes and procedures by which capital project close outs and look. backs are performed. . Maintaining and performing the established processes and procedures of capital approval, tracking, monitoring, close. outs and look. backs. . Champion change by supporting continuous improvement of business processes and systems Note: This position agrees with and has adopted the major elements of the HOIMS program with respect to the way in which all work is undertaken in this Business Unit.","Husky Energy is one of Canada’s largest integrated energy companies. Based in Calgary, Alberta, its common shares are publicly traded

In [4]:
df[["Position Title:","Department:"]].head()

,Position Title:,Department:
0,Environmental Engineer/Specialist,Technologists (US)
1,"Sr. Staff Engineer, Capital Efficiency",Engineers
2,Machinist (Millwright/Pump Mechanic),Lima Union (US)
3,Coker Utility Operator,Lima Union (US)
4,Maintenance Planner - Sunrise,Maintenance Planners


In [5]:
df.fillna("EMPTY",inplace=True)

In [6]:
df['result'] = df.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [7]:
#df['result']

0       Environmental Engineer/Specialist,Technologists (US), Manage the permitting actions for the RCRA Hazardous Waste Installation and Operation Permit. .  Manage the Corrective Actions Program/Site Remediation (including the Soil Management Program. .  Supervise the waste handling contractors (including the internal movement of waste in containers/tanks), and possibly the Leak Detection and Repair (LDAR) contractors. .  Function as the Oil Spill Director for the site, with responsibility for keeping the Integrated Contingency Plan current and conducting all identified drills. .  Oversee the Stormwater Management Plan and the Biosolids Management Plan. .  Manage the permitting process for the Deepwell Injection Project. .  Support refinery compliance with applicable environmental regulation and permit requirements for the waste management program for the refinery. .  Perform emissions and discharge calculations, completing regulatory compliance reports and coordinating environmental

In [8]:
df['pstdt'] = df.apply(lambda x: ','.join(x[["Position Title:","Department:"]].dropna().astype(str)), axis=1)

In [9]:
df["pstdt"].head()

0    Environmental Engineer/Specialist,Technologists (US)
1        Sr. Staff Engineer, Capital Efficiency,Engineers
2    Machinist (Millwright/Pump Mechanic),Lima Union (US)
3                  Coker Utility Operator,Lima Union (US)
4      Maintenance Planner - Sunrise,Maintenance Planners
Name: pstdt, dtype: object

In [10]:
from gensim.parsing.preprocessing import remove_stopwords,strip_punctuation,strip_punctuation2,preprocess_string,strip_non_alphanum
from gensim.summarization.textcleaner import split_sentences
from gensim import corpora, models, similarities

In [11]:
CUSTOM_FILTERS=[remove_stopwords,strip_punctuation,strip_punctuation2,strip_non_alphanum]

In [12]:
 S=[]
for one in df["pstdt"]:
    S.append(preprocess_string(one,CUSTOM_FILTERS))

In [13]:
(S[0])

['Environmental', 'Engineer', 'Specialist', 'Technologists', 'US']

In [14]:
dictionary=corpora.Dictionary(S)

In [15]:
dictionary.token2id

{'Engineer': 0,
 'Environmental': 1,
 'Specialist': 2,
 'Technologists': 3,
 'US': 4,
 'Capital': 5,
 'Efficiency': 6,
 'Engineers': 7,
 'Sr': 8,
 'Staff': 9,
 'Lima': 10,
 'Machinist': 11,
 'Mechanic': 12,
 'Millwright': 13,
 'Pump': 14,
 'Union': 15,
 'Coker': 16,
 'Operator': 17,
 'Utility': 18,
 'Maintenance': 19,
 'Planner': 20,
 'Planners': 21,
 'Sunrise': 22,
 'Process': 23,
 'Co': 24,
 'Engineering': 25,
 'Exploitation': 26,
 'Internship': 27,
 'Student': 28,
 'Students': 29,
 'Subsurface': 30,
 'Technical': 31,
 'op': 32,
 'Logistics': 33,
 'Operations': 34,
 'Representative': 35,
 'Reps': 36,
 'Supply': 37,
 'Transportation': 38,
 'Analysts': 39,
 'Economic': 40,
 'Economics': 41,
 'Manager': 42,
 'Planning': 43,
 'Commodity': 44,
 'Light': 45,
 'Products': 46,
 'Trader': 47,
 'Trading': 48,
 'Development': 49,
 'Thermal': 50,
 'Lloyd': 51,
 'Relief': 52,
 'Summer': 53,
 '5': 54,
 'Operators': 55,
 'Plant': 56,
 'Technicians': 57,
 'Worker': 58,
 'Trainers': 59,
 'Analyst': 6

In [16]:
corpus = [dictionary.doc2bow(text) for text in S]

In [17]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(0, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(4, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)],
 [(4, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1)],
 [(19, 2), (20, 1), (21, 1), (22, 1)],
 [(0, 1), (7, 1), (23, 1)],
 [(24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1)],
 [(33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)],
 [(4, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2)],
 [(4, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)],
 [(0, 1), (7, 1), (49, 1), (50, 1)],
 [(15, 2), (51, 2), (52, 1), (53, 1)],
 [(17, 1), (23, 1), (54, 1), (55, 1), (56, 2), (57, 1)],
 [(15, 1), (51, 1), (52, 1), (53, 1), (58, 1)],
 [(2, 1), (4, 1), (31, 1), (34, 1), (49, 1), (59, 1)],
 [(4, 2), (28, 1), (29, 1), (33, 1), (53, 1)],
 [(4, 2), (28, 1), (29, 1), (53, 1), (60, 1), (61, 1)],
 [(15, 2), (51, 2), (62, 1), (63, 1)],
 [(17, 1), (55, 1), (64, 2), (65, 1), (66, 1)],
 [(17, 1), (55, 1), (64

In [18]:
tfidf = models.TfidfModel(corpus)

In [19]:
corpus_tfidf = tfidf[corpus]

In [20]:
for doc in corpus_tfidf:
    print(doc)

[(0, 0.321960063265196), (1, 0.5671421719907045), (2, 0.4645752340735951), (3, 0.5031981839650832), (4, 0.3250429417040829)]
[(0, 0.2147209014375219), (5, 0.57575468157542), (6, 0.6661794781631408), (7, 0.19168765287939102), (8, 0.24482137647140043), (9, 0.28622494887827116)]
[(4, 0.15858179743385784), (10, 0.2994436724264756), (11, 0.48733940480875393), (12, 0.43729909540560463), (13, 0.37875568635495155), (14, 0.48733940480875393), (15, 0.2747717977138864)]
[(4, 0.20119909022989788), (10, 0.3799162037650313), (15, 0.3486140062444781), (16, 0.554818910982907), (17, 0.27928986893384755), (18, 0.554818910982907)]
[(19, 0.6254583324316201), (20, 0.44036451167940993), (21, 0.41350382889594817), (22, 0.49385782846240517)]
[(0, 0.5467359470807479), (7, 0.48808723202568266), (23, 0.680331285553755)]
[(24, 0.1754438605218049), (25, 0.24195109233105375), (26, 0.6847389985217669), (27, 0.2430815248148041), (28, 0.14762608848156492), (29, 0.13515573371506476), (30, 0.5353246699495736), (31, 0.15

[(35, 0.49454207623545515), (36, 0.4154599024945954), (87, 0.47776984024035946), (201, 0.5954470455116961)]
[(94, 0.5307528061797832), (111, 0.45095803904802123), (119, 0.387506819513978), (288, 0.6039675244418446)]
[(98, 0.3558704024086813), (119, 0.3430013318761395), (184, 0.8693137195631262)]
[(39, 0.21521011047065383), (60, 0.20692292029504203), (74, 0.463112900322308), (82, 0.6743062321644137), (185, 0.49163508848237475)]
[(0, 0.4666013820585171), (4, 0.47106925091689206), (7, 0.41654875309426875), (9, 0.621983960723498)]
[(117, 0.8250449604540988), (118, 0.4101415121728388), (119, 0.38869622228917)]
[(0, 0.27364742383157786), (7, 0.24429308949240813), (34, 0.3416214751240735), (200, 0.5418312726450791), (500, 0.6746492522596175)]
[(39, 0.3160690326684821), (60, 0.30389802371069174), (61, 0.7307230705009569), (125, 0.5232496077498762)]
[(4, 0.1910099065322471), (39, 0.1887511949961025), (118, 0.30885233499544873), (280, 0.9124135231143523)]
[(60, 0.16752310104266954), (96, 0.24881

[(24, 0.2560810571527631), (25, 0.35315622512593703), (28, 0.2154777299658657), (29, 0.19727577281457248), (31, 0.21908046285871768), (32, 0.2673996840726642), (49, 0.5526330030560577), (50, 0.5472227278322649)]
[(24, 0.22024933508453856), (28, 0.18532736188368862), (29, 0.16967228374404286), (31, 0.18842598828331522), (32, 0.22998422168995836), (43, 0.4956143845188772), (176, 0.7445297858596569)]
[(24, 0.3059558253405408), (25, 0.4219375128090768), (28, 0.2574445273196627), (29, 0.23569752703406588), (31, 0.261748934400646), (32, 0.3194788866692598), (49, 0.6602647163377401)]
[(24, 0.1655552662772671), (25, 0.22831393129293845), (28, 0.1393053955569983), (29, 0.1275379110876097), (31, 0.14163454637367184), (32, 0.1728727083186729), (49, 0.3572747824138081), (376, 0.5382365029539623), (619, 0.6461448516553316)]
[(24, 0.20431737122632287), (28, 0.1719215151403326), (29, 0.15739886329846728), (31, 0.17479599918879157), (32, 0.213348074722623), (405, 0.5236788339561537), (406, 0.744051368

In [21]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)  # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] 

In [22]:
lsi.print_topics()

[(0,
  '0.357*"Technical" + 0.347*"op" + 0.341*"Student" + 0.341*"Co" + 0.323*"Students" + 0.297*"Engineering" + 0.272*"Internship" + 0.155*"Non" + 0.136*"Engineer" + 0.131*"Project"'),
 (1,
  '0.421*"Engineers" + 0.415*"Engineer" + 0.323*"Business" + 0.278*"Analyst" + 0.278*"Financial" + 0.268*"Analysts" + 0.252*"Staff" + 0.176*"Sr" + -0.145*"op" + -0.137*"Co"'),
 (2,
  '0.407*"Engineer" + -0.391*"Business" + 0.389*"Engineers" + -0.384*"Financial" + -0.352*"Analyst" + -0.336*"Analysts" + 0.240*"Staff" + 0.132*"Project" + -0.093*"Accountants" + 0.084*"Production"'),
 (3,
  '0.774*"Administrative" + 0.446*"Assistants" + 0.426*"Assistant" + 0.053*"Executive" + 0.049*"US" + 0.039*"Administrator" + 0.038*"Sr" + -0.034*"Financial" + 0.028*"Safety" + 0.028*"Field"'),
 (4,
  '0.543*"Plant" + 0.330*"Operators" + 0.311*"Process" + 0.296*"Operator" + 0.283*"Technicians" + 0.197*"Field" + -0.173*"Business" + 0.154*"Foreman" + 0.143*"Financial" + 0.138*"Superintendents"'),
 (5,
  '0.680*"Financial

In [23]:
ldi = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

In [24]:
from sklearn import cluster

In [36]:
k_means = cluster.KMeans(n_clusters=3)

ValueError: could not convert string to float: 'Environmental Engineer/Specialist,Technologists (US)'